## Tratamento dos dados

### Importando bibliotecas

In [22]:
import numpy as np
from sklearn.metrics import adjusted_rand_score
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

### Geração de dados sintéticos

In [28]:
# Gerar dados com ruído nas 7 configurações acima
# Ruído de 5%, 10% e 15% em relação ao total de dados
# Intervalo de ruído: [-100, 50]

def generate_data(config_params, noise_percent):
    data_all = []
    for class_idx, param in enumerate(config_params, start=1):
        mu = np.array(param['mu'])
        sigma_diag = np.diag(param['sigma'])
        n = param['n']
        n_noise = int(n * noise_percent / 100)
        n_signal = n - n_noise

        real_data = np.random.multivariate_normal(mu, sigma_diag, n_signal)
        labels_real = np.full((n_signal,), class_idx)

        noise_data = np.random.uniform(noise_range[0], noise_range[1], size=(n_noise, 2))
        labels_noise = np.full((n_noise,), 0)

        data = np.vstack([real_data, noise_data])
        labels = np.concatenate([labels_real, labels_noise])

        df = pd.DataFrame(data, columns=['x1', 'x2'])
        df['class'] = labels
        data_all.append(df)

    return pd.concat(data_all, ignore_index=True)

configurations = {
    1: {'mu': [[5, 0], [15, 5], [18, 14]],
        'sigma': [[81, 9], [9, 100], [25, 36]],
        'n': [200, 100, 50]},
    2: {'mu': [[0, 0], [30, 0], [12, 25]],
        'sigma': [[100, 100], [49, 49], [16, 16]],
        'n': [200, 100, 50]},
    3: {'mu': [[0, 0], [15, 5], [15, -5]],
        'sigma': [[100, 4], [100, 4], [100, 4]],
        'n': [100, 100, 100]},
    4: {'mu': [[0, 0], [15, 0], [-15, 0]],
        'sigma': [[16, 16], [16, 16], [16, 16]],
        'n': [100, 100, 100]},
    5: {'mu': [[5, 0], [15, 5], [10, -7], [3, 15]],
        'sigma': [[81, 9], [9, 100], [49, 16], [25, 25]],
        'n': [50, 50, 50, 50]},
    6: {'mu': [[5, 0], [15, 5], [12, -12], [7, 17]],
        'sigma': [[81, 9], [9, 100], [16, 16], [25, 25]],
        'n': [50, 50, 50, 50]},
    7: {'mu': [[0, 0], [18, 0], [-18, 0], [0, -12]],
        'sigma': [[12, 12], [20, 20], [16, 16], [81, 20]],
        'n': [50, 50, 50, 50]},
}

noise_range = (-100, 50)

noise_levels = [5, 10, 15]

all_datasets = {}  # Key: (config_id, noise_level), Value: DataFrame

for row_idx, noise in enumerate(noise_levels):
    for col_idx, config_id in enumerate(sorted(configurations.keys())):
        config = configurations[config_id]
        config_params = [{'mu': mu, 'sigma': sigma, 'n': n}
                         for mu, sigma, n in zip(config['mu'], config['sigma'], config['n'])]

        df = generate_data(config_params, noise)

        # Store the data
        all_datasets[(config_id, noise)] = df

### Escolhendo e verificando um dos conjuntos gerados

In [29]:
for (config_id, noise_level), df in all_datasets.items():
    labels = df["class"].values
    num_clusters = len(df['class'].unique())
    df.drop("class", axis=1, inplace=True)
    dados = df.to_numpy()
    def inicializacao_matriz_pertinencia(num_amostras, num_clusters):
        matriz_pertinencia = np.random.rand(num_amostras, num_clusters) # gera uma matriz inicial aleatória com valores entre 0 e 1
        matriz_pertinencia = matriz_pertinencia / matriz_pertinencia.sum(axis=1, keepdims=True) # normalização da matriz pra garantir que a soma dos graus dê um
        return matriz_pertinencia
    def atualizacao_centroides(dados, matriz_pertinencia, m):
        matriz_pertinencia_m = matriz_pertinencia ** m # preparação dos graus de pertinência
        centroides = np.dot(matriz_pertinencia_m.T, dados) / np.sum(matriz_pertinencia_m.T, axis=1, keepdims=True) # fórmula para o cálculo dos centroides
        return centroides
    def atualizacao_matriz_pertinencia(dados, centroides, m):
        # dados[:, np.newaxis] - centroides cria uma matriz de diferenças entre os pontos de dados e os centroides
        # np.linalg.norm(..., axis=2) calcula a norma (distância euclidiana) das diferenças
        # ** 2 para a distância ser a quadrada
        matriz_distancias = np.linalg.norm(dados[:, np.newaxis] - centroides, axis=2) ** 2
        matriz_distancias = np.fmax(matriz_distancias, np.finfo(np.float64).eps) # evita que matriz_distancias seja 0, np.finfo... é o menor número maior que zero aaqui
        matriz_distancias_inversa = 1 / matriz_distancias
        potencia = 1 / (m-1)
        matriz_pertinencia_atualizada = matriz_distancias_inversa ** potencia/ np.sum(matriz_distancias_inversa ** potencia, axis=1, keepdims=True) # fórmula para atualizar os graus de pertinência
        return matriz_pertinencia_atualizada
    def fcm(dados, num_clusters, m=2, max_iter=10**6, erro=1e-9):
        num_amostras = dados.shape[0]
        matriz_pertinencia = inicializacao_matriz_pertinencia(num_amostras, num_clusters)
        for _ in range(max_iter): # primeiro critério de parada
            centroides = atualizacao_centroides(dados, matriz_pertinencia, m)
            nova_matriz_pertinencia = atualizacao_matriz_pertinencia(dados, centroides, m)
            if np.linalg.norm(nova_matriz_pertinencia - matriz_pertinencia) < erro: # segundo critério de parada
                break
            matriz_pertinencia = nova_matriz_pertinencia
        return centroides, matriz_pertinencia
    def indice_rand(labels, predicted_labels):
        return adjusted_rand_score(labels, predicted_labels)
    def simulacao_monte_carlo(dados, labels, num_clusters, num_trials):
        indices_rand = []
        for _ in range(num_trials):
            centroides, matriz_pertinencia = fcm(dados, num_clusters)
            predicted_labels = np.argmax(matriz_pertinencia, axis=1)
            #print(predicted_labels)
            idx_rand = indice_rand(labels, predicted_labels)
            indices_rand.append(idx_rand)
        mean_rand_index = np.mean(indices_rand)
        std_rand_index = np.std(indices_rand)
        return mean_rand_index, std_rand_index
    num_trials = 100
    media_indice_rand, dp_indice_rand = simulacao_monte_carlo(dados, labels, num_clusters, num_trials)

    print((config_id, noise_level))
    print(f"Mean Rand Index: {media_indice_rand:.4f}")
    print(f"Standard Deviation of Rand Index: {dp_indice_rand:.4f}")

(1, 5)
Mean Rand Index: 0.3193
Standard Deviation of Rand Index: 0.0000
(2, 5)
Mean Rand Index: 0.4316
Standard Deviation of Rand Index: 0.0000
(3, 5)
Mean Rand Index: 0.1267
Standard Deviation of Rand Index: 0.0000
(4, 5)
Mean Rand Index: 0.8159
Standard Deviation of Rand Index: 0.0000
(5, 5)
Mean Rand Index: 0.4126
Standard Deviation of Rand Index: 0.0553
(6, 5)
Mean Rand Index: 0.4824
Standard Deviation of Rand Index: 0.0130
(7, 5)
Mean Rand Index: 0.6861
Standard Deviation of Rand Index: 0.0089
(1, 10)
Mean Rand Index: 0.2919
Standard Deviation of Rand Index: 0.0009
(2, 10)
Mean Rand Index: 0.5541
Standard Deviation of Rand Index: 0.0000
(3, 10)
Mean Rand Index: 0.1825
Standard Deviation of Rand Index: 0.0000
(4, 10)
Mean Rand Index: 0.7975
Standard Deviation of Rand Index: 0.0000
(5, 10)
Mean Rand Index: 0.2901
Standard Deviation of Rand Index: 0.0000
(6, 10)
Mean Rand Index: 0.5066
Standard Deviation of Rand Index: 0.0210
(7, 10)
Mean Rand Index: 0.6930
Standard Deviation of Rand

### Resultados

Configuração 1:  
Mean Rand Index: 0.2943  
Standard Deviation of Rand Index: 0.0000  

Configuração 2:  
Mean Rand Index: 0.6129  
Standard Deviation of Rand Index: 0.0000  

Configuração 3:  
Mean Rand Index: 0.1285  
Standard Deviation of Rand Index: 0.0000  

Configuração 4:  
Mean Rand Index: 0.8470  
Standard Deviation of Rand Index: 0.0000  

Configuração 5:  
Mean Rand Index: 0.4291  
Standard Deviation of Rand Index: 0.0000  

Configuração 6:  
Mean Rand Index: 0.4814  
Standard Deviation of Rand Index: 0.0000  

Configuração 7:  
Mean Rand Index: 0.7653  
Standard Deviation of Rand Index: 0.0000

In [30]:
# Parâmetros das configurações
params_config_12 = [
    {'mu': [-16, -5], 'sigma': [20, 20], 'n': 50},
    {'mu': [-8, 8], 'sigma': [13, 13], 'n': 100},
    {'mu': [0, 0], 'sigma': [6, 6], 'n': 200},
]

params_config_13 = [
    {'mu': [7, -6], 'sigma': [50, 5], 'n': 100},
    {'mu': [0, 0], 'sigma': [2, 50], 'n': 100},
    {'mu': [12, 0], 'sigma': [50, 5], 'n': 100},
]

# Faixa para ruído e semente aleatória
noise_range = [-100, 50]
np.random.seed(42)

# Função de geração de dados com ruído
def generate_data(config_params, noise_percent):
    data_all = []
    for class_idx, param in enumerate(config_params, start=1):
        mu = np.array(param['mu'])
        sigma_diag = np.diag(param['sigma'])
        n = param['n']
        n_noise = int(n * noise_percent / 100)
        n_signal = n - n_noise

        real_data = np.random.multivariate_normal(mu, sigma_diag, n_signal)
        labels_real = np.full((n_signal,), class_idx)

        noise_data = np.random.uniform(noise_range[0], noise_range[1], size=(n_noise, 2))
        labels_noise = np.full((n_noise,), 0)

        data = np.vstack([real_data, noise_data])
        labels = np.concatenate([labels_real, labels_noise])

        df = pd.DataFrame(data, columns=['x1', 'x2'])
        df['class'] = labels
        data_all.append(df)

    return pd.concat(data_all, ignore_index=True)

# Configurações e títulos
configs = [
    (params_config_12, 10),
    (params_config_12, 20),
    (params_config_12, 30),
    (params_config_13, 10),
    (params_config_13, 20),
    (params_config_13, 30),
]

titles = [
    "Configuração 12 - 10% ruído",
    "Configuração 12 - 20% ruído",
    "Configuração 12 - 30% ruído",
    "Configuração 13 - 10% ruído",
    "Configuração 13 - 20% ruído",
    "Configuração 13 - 30% ruído",
]

# Geração dos DataFrames separadamente
dfs_por_config = {}

for (params, noise), title in zip(configs, titles):
    df = generate_data(params, noise)
    df['classe_legenda'] = df['class'].replace(0, 'ruído')
    dfs_por_config[title] = df

In [31]:
for nome_config, df in dfs_por_config.items():
    labels = df["class"].values
    df.drop("class", axis=1, inplace=True)
    df.drop("classe_legenda", axis=1, inplace=True)
    dados = df.to_numpy()
    num_clusters = 4
    num_trials = 100
    media_indice_rand, dp_indice_rand = simulacao_monte_carlo(dados, labels, num_clusters, num_trials)
    print("\n")
    print(f"Resultados de Monte Carlo para {nome_config} ({num_trials} tentativas)")
    print(f"Média do Índice Rand: {media_indice_rand:.4f}")
    print(f"Desvio Padrão do Índice Rand: {dp_indice_rand:.4f}")



Resultados de Monte Carlo para Configuração 12 - 10% ruído (100 tentativas)
Média do Índice Rand: 0.8424
Desvio Padrão do Índice Rand: 0.0246


Resultados de Monte Carlo para Configuração 12 - 20% ruído (100 tentativas)
Média do Índice Rand: 0.6343
Desvio Padrão do Índice Rand: 0.0711


Resultados de Monte Carlo para Configuração 12 - 30% ruído (100 tentativas)
Média do Índice Rand: 0.5150
Desvio Padrão do Índice Rand: 0.0231


Resultados de Monte Carlo para Configuração 13 - 10% ruído (100 tentativas)
Média do Índice Rand: 0.2731
Desvio Padrão do Índice Rand: 0.0000


Resultados de Monte Carlo para Configuração 13 - 20% ruído (100 tentativas)
Média do Índice Rand: 0.2311
Desvio Padrão do Índice Rand: 0.0005


Resultados de Monte Carlo para Configuração 13 - 30% ruído (100 tentativas)
Média do Índice Rand: 0.1913
Desvio Padrão do Índice Rand: 0.0093


Os centroides das três classes estão localizados próximos uns dos outros:
$\mu_1 = \begin{bmatrix} 20 \\ 20 \end{bmatrix},$
$\mu_2 = \begin{bmatrix} 23 \\ 23 \end{bmatrix},$
$\mu_3 = \begin{bmatrix} 26 \\ 20 \end{bmatrix}$

As classes apresentam diferentes formas e orientações devido às suas matrizes de covariância:
$\Sigma_1 = \begin{bmatrix} 10 & 9 \\ 9 & 10 \end{bmatrix},$
$\Sigma_2 = \begin{bmatrix} 10 & -9 \\ -9 & 10 \end{bmatrix},$
$\Sigma_3 = \begin{bmatrix} 12 & 0 \\ 0 & 1 \end{bmatrix}$

- $\Sigma_1$ e $\Sigma_2$ geram distribuições elípticas com inclinação forte nas diagonais principais e secundárias, respectivamente.
- $\Sigma_3$ resulta em uma distribuição fortemente alongada no eixo $x$.

Cada classe possui $5\%$ de outliers, gerados a partir dos mesmos centros e covariâncias, mas com deslocamentos adicionais direcionados para regiões distantes dos centros originais. Os deslocamentos aplicados foram:
$\Delta_1 = \begin{bmatrix} -10 \\ 5 \end{bmatrix},$
$\Delta_2 = \begin{bmatrix} 10 \\ -10 \end{bmatrix},$
$\Delta_3 = \begin{bmatrix} 6 \\ 10 \end{bmatrix}$

In [32]:
def config1_outliers():
    np.random.seed(42)
    n = 150
    frac_outlier = 0.05

    # Covariâncias exageradas para formas mais elípticas e inclinadas
    covs = [
        [[10, 9], [9, 10]],     # fortemente inclinado (diagonal)
        [[10, -9], [-9, 10]],   # diagonal oposta
        [[12, 0], [0, 1]]       # fortemente alongado no eixo x
    ]

    mus = [[20, 20], [23, 23], [26, 20]]  # centroides próximos!
    deslocamentos_outliers = [[-10, 5], [10, -10], [6, 10]]

    dados, rotulos, outlier_flags = [], [], []

    for i, (mu, cov, desloc) in enumerate(zip(mus, covs, deslocamentos_outliers)):
        classe = np.random.multivariate_normal(mu, cov, size=n)
        n_outliers = int(n * frac_outlier)

        for j, ponto in enumerate(classe):
            if j < n_outliers:
                outlier = ponto + desloc + np.random.normal(0, 1.8, size=2)
                dados.append(outlier)
                outlier_flags.append(1)
            else:
                dados.append(ponto)
                outlier_flags.append(0)
            rotulos.append(f'Classe {i+1}')

    # Garante que tudo fique no primeiro quadrante
    dados = np.array(dados)
    dados -= np.min(dados, axis=0)
    dados += 1

    df = pd.DataFrame(dados, columns=["x1", "x2"])
    df["Classe"] = rotulos
    df["Outlier"] = outlier_flags
    return df

# 🔍 Visualização
df = config1_outliers()

In [33]:
df.drop("Outlier", axis=1, inplace=True)
df['Classe'].replace({'Classe 1': 0, 'Classe 2': 1, 'Classe 3': 2}, inplace=True)
labels = df["Classe"].values
df.drop("Classe", axis=1, inplace=True)
dados = df.to_numpy()
num_clusters = 3
num_trials = 100
media_indice_rand, dp_indice_rand = simulacao_monte_carlo(dados, labels, num_clusters, num_trials)
print(f"Resultados de Monte Carlo para dados com outliers ({num_trials} tentativas)")
print(f"Média do Índice Rand: {media_indice_rand:.4f}")
print(f"Desvio Padrão do Índice Rand: {dp_indice_rand:.4f}")

Resultados de Monte Carlo para dados com outliers (100 tentativas)
Média do Índice Rand: 0.3229
Desvio Padrão do Índice Rand: 0.0000
